## Visualise stokes cartesian model (flow etc)

In [1]:
# to fix trame issue
import nest_asyncio
nest_asyncio.apply()
import os

In [2]:
import petsc4py
import underworld3 as uw
import numpy as np
import sympy

[Lyrebird.local:16520] shmem: mmap: an error occurred while determining whether or not /var/folders/tx/95gr762j29z4tt5d1dnqlgth0000gn/T//ompi.Lyrebird.501/jf.0/2560229376/sm_segment.Lyrebird.501.989a0000.0 could be created.


In [3]:
ls -trl output/cartesian* 

output/cartesian:
total 0
drwxr-xr-x@  294 lmoresi  staff    9408 Aug 26 21:59 Ra1e3.0/
drwxr-xr-x@  909 lmoresi  staff   29088 Aug 27 16:34 Ra1e4.0/
drwxr-xr-x@ 2009 lmoresi  staff   64288 Aug 27 18:41 Ra1e5.0/
drwxr-xr-x  7517 lmoresi  staff  240544 Aug 28 21:19 Ra1e7.0/
drwxr-xr-x  6758 lmoresi  staff  216256 Aug 29 11:44 Ra1e8.0/
drwxr-xr-x@ 6818 lmoresi  staff  218176 Aug 29 11:56 Ra1e6.0/

output/cartesian_2x1:
total 0
drwxr-xr-x  6259 lmoresi  staff  200288 Aug 31 10:40 Ra1e6.0/

output/cartesian_1x1:
total 0
drwxr-xr-x@  7022 lmoresi  staff  224704 Sep  6 19:25 Ra1e6.0/
drwxr-xr-x@  5516 lmoresi  staff  176512 Sep  6 23:26 Ra1e8.0/
drwxr-xr-x@ 16529 lmoresi  staff  528928 Sep  7 05:42 Ra1e7.0/


In [4]:
ls -trl output/cartesian_1x1/Ra1e8.0/ | tail

-rw-r--r--@   1 lmoresi  staff   61432 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.U.00299.h5
-rw-r--r--@   1 lmoresi  staff   28360 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.P.00299.h5
-rw-r--r--@   1 lmoresi  staff  101480 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.T.00299.h5
-rw-r--r--@   1 lmoresi  staff  101480 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.W_14_0.00299.h5
-rw-r--r--@   1 lmoresi  staff    3751 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.00299.xdmf
-rw-r--r--@   1 lmoresi  staff   61432 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.U.00300.h5
-rw-r--r--@   1 lmoresi  staff   28360 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.P.00300.h5
-rw-r--r--@   1 lmoresi  staff  101480 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.T.00300.h5
-rw-r--r--@   1 lmoresi  staff  101480 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.W_14_0.00300.h5
-rw-r--r--@   1 lmoresi  staff    3751 Sep  6 23:26 Ra1e8.0_visc4_res15.mesh.00300.xdmf


In [20]:
# This is a bit repetitive, sorry 

checkpoint_dir = "output/cartesian_1x1/Ra1e8.0"
name = "Ra1e8.0_visc4"
resolution = 25

checkpoint_base = f"{name}_res{resolution}"
meshfile = os.path.join(checkpoint_dir, checkpoint_base) + ".mesh.00000.h5"
image_dir = os.path.join(checkpoint_dir,"images")
os.makedirs(image_dir, exist_ok=True)

In [21]:
mesh = uw.discretisation.Mesh(meshfile, 
                              coordinate_system_type=uw.coordinates.CoordinateSystemType.CARTESIAN)
    
x = mesh.N.x
y = mesh.N.y

v_soln = uw.discretisation.MeshVariable("U", mesh, mesh.dim, degree=2)
t_soln = uw.discretisation.MeshVariable("T", mesh, 1, degree=3)
flux = uw.discretisation.MeshVariable(r"dTdz", mesh, 1, degree=1, continuous=False)

In [28]:
flux_solver = uw.systems.Projection(mesh, flux)

flux_fn = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_1) # + v_soln.sym[1] * t_soln.sym[0]
flux_solver.uw_function = flux_fn
flux_solver.smoothing = 0.0


sample_points = np.zeros((100,2))
sample_points[:,0] = np.linspace(0,1,100)
sample_points[:,1] = 1

sample_points_surf = sample_points.copy()

sample_points[:,1] = 0
sample_points_base = sample_points.copy()



In [29]:
flux_solver.view()

**Class**: <class 'underworld3.systems.solvers.SNES_Projection'>

**Poisson system solver**

Primary problem: 

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

This solver is formulated in 2 dimensions

In [44]:
steps = range(1450,1800, 1)
calc_flux = True
add_streamlines = False
cmap = "RdBu_r"

In [45]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pl = pv.Plotter(window_size=[750, 750])

for step in steps:

    try:
        v_soln.read_timestep(checkpoint_base, "U", step, outputPath=checkpoint_dir)
        t_soln.read_timestep(checkpoint_base, "T", step, outputPath=checkpoint_dir)
        if not calc_flux:
            print(f"Plotting {step:05d}")

    except:
        print(f"Failed to read step {step} correctly")
        continue

    pl.clear()

    pvmesh = vis.mesh_to_pv_mesh(mesh)
    pvmesh.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh, t_soln.sym[0])
    pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)

    pvmesh_v = vis.meshVariable_to_pv_mesh_object(v_soln, alpha=None)
    pvmesh_v.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh_v, v_soln.sym)

    pvmesh_t = vis.meshVariable_to_pv_mesh_object(t_soln, alpha=None)
    pvmesh_t.point_data["T"] = vis.scalar_fn_to_pv_points(pvmesh_t, t_soln.sym[0])
    
    # point sources at cell centres
    skip = 19
    points = np.zeros((mesh._centroids[::skip].shape[0], 3))
    points[:, 0] = mesh._centroids[::skip, 0]
    points[:, 1] = mesh._centroids[::skip, 1]
    point_cloud = pv.PolyData(points)


    
    pl.add_mesh(
        pvmesh_t,
        cmap=cmap,
        edge_color="Black",
        edge_opacity=0.1,
        show_edges=False,
        scalars="T",
        use_transparency=False,
        opacity=1.0,
        # clim=[0.0,1.0],
        show_scalar_bar=False
    )



    # pl.add_mesh(pvmesh, cmap="RdBu_r", scalars="T", opacity=0.5, show_edges=True, edge_color="Black",
    #     edge_opacity=0.5, show_scalar_bar=True)   

    if add_streamlines:
        pvstream = pvmesh.streamlines_from_source(
                point_cloud,
                vectors="V",
                integration_direction="both",
                max_time=1,
                surface_streamlines=True,
            )
    
        pl.add_mesh(pvstream,
                    opacity=0.5,
                    show_scalar_bar=False)

    # pl.camera_position = [(3.0, 0.5, 6.750994846396983),
    #                       (3.0, 0.5, 0.0),
    #                       (0.0, 1.0, 0.0)]

    imagefile = os.path.join(image_dir, checkpoint_base) + f"_V_T_{step}.png"
    
    pl.screenshot(filename=imagefile,
                  window_size=(1000, 1000),
                  return_img=False)

    if calc_flux:
        flux_solver.solve()
        surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
        basal_flux = uw.function.evaluate(flux.sym, sample_points_base)
    
        # Heat flux: Upper, Lower, Mean
        print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}")




01450,    7.3276,  13.6851,  10.5063
01451,    7.3353,  13.6430,  10.4891
01452,    7.3432,  13.6123,  10.4778
01453,    7.3486,  13.5973,  10.4729
01454,    7.3421,  13.6011,  10.4716
01455,    7.3297,  13.6033,  10.4665
01456,    7.3159,  13.5921,  10.4540
01457,    7.3034,  13.5625,  10.4330
01458,    7.2932,  13.5192,  10.4062
01459,    7.2835,  13.4734,  10.3785
01460,    7.2782,  13.4263,  10.3523
01461,    7.2785,  13.3781,  10.3283
01462,    7.2853,  13.3278,  10.3066
01463,    7.2984,  13.2783,  10.2883
01464,    7.3059,  13.2471,  10.2765
01465,    7.3093,  13.2254,  10.2673
01466,    7.3100,  13.2122,  10.2611
01467,    7.3044,  13.2121,  10.2583
01468,    7.2951,  13.2182,  10.2567
01469,    7.2819,  13.2262,  10.2540
01470,    7.2697,  13.2288,  10.2492
01471,    7.2605,  13.2212,  10.2408
01472,    7.2553,  13.1988,  10.2270
01473,    7.2518,  13.1712,  10.2115
01474,    7.2499,  13.1478,  10.1988
01475,    7.2468,  13.1307,  10.1888
01476,    7.2419,  13.1211,  10.1815
0

In [47]:
pl.show(jupyter_backend="trame")


A view with name (P_0x316480f50_8) is already registered
 => returning previous one


Widget(value='<iframe src="http://localhost:51965/index.html?ui=P_0x316480f50_8&reconnect=auto" class="pyvista…

In [12]:
## Calculate heat flux, evaluate at surface — proxy for boundary layer thickness

In [13]:
t_soln.stats()

(2539,
 0.6023444526269115,
 7.818744591315606e-10,
 0.9999999953965953,
 1529.3525652197284,
 31.51263814931343,
 0.6253935706479435)

In [14]:
I = uw.maths.Integral(mesh, t_soln.sym[0])
mean = I.evaluate()
mean

0.6004368703200991

In [15]:
I.fn = (t_soln.sym[0]-mean)**2
np.sqrt(I.evaluate())
l2 = np.sqrt(I.evaluate())
l2

0.1536791235489102

In [16]:
! open output/cartesian_1x1/Ra1e6.0/images

In [17]:
0/0

ZeroDivisionError: division by zero

In [ ]:
flux_solver = uw.systems.Projection(mesh, flux)

# Conductive flux only !
flux_fn = -mesh.vector.gradient(t_soln.sym[0]).dot(mesh.CoordinateSystem.unit_e_1)  + v_soln.sym[1] * t_soln.sym[0]
flux_solver.uw_function = flux_fn
flux_solver.smoothing = 0.0
flux_solver.solve()

surf_flux = uw.function.evaluate(flux.sym, sample_points_surf)
basal_flux = uw.function.evaluate(flux.sym, sample_points_base)

# Heat flux: Upper, Lower, Mean
print(f"{step:05d},  {surf_flux.mean():8.4f}, {basal_flux.mean():8.4f}, {((surf_flux.mean() + basal_flux.mean())/2):8.4f}")



In [ ]:
import mpi4py
import pyvista as pv
import underworld3.visualisation as vis

pvmesh = vis.mesh_to_pv_mesh(mesh)
pvmesh.point_data["dTdz"] = vis.scalar_fn_to_pv_points(pvmesh, flux.sym[0])
pvmesh.point_data["V"] = vis.vector_fn_to_pv_points(pvmesh, v_soln.sym)
pvmesh.point_data["V"] -= pvmesh.point_data["V"].mean()

velocity_points = vis.meshVariable_to_pv_cloud(v_soln)
velocity_points.point_data["V"] = vis.vector_fn_to_pv_points(velocity_points, v_soln.sym)

# point sources at cell centres
skip = 8
points = np.zeros((mesh._centroids[::skip].shape[0], 3))
points[:, 0] = mesh._centroids[::skip, 0]
points[:, 1] = mesh._centroids[::skip, 1]
point_cloud = pv.PolyData(points)

pvstream = pvmesh.streamlines_from_source(
    point_cloud,
    vectors="V",
    integration_direction="both",
    max_time=1.5,
)

pl = pv.Plotter(window_size=[1000, 1000])

pl.add_mesh(
    pvmesh,
    cmap="RdYlBu_r",
    edge_color="Black",
    show_edges=False,
    scalars="dTdz",
    use_transparency=False,
    opacity=1.0,
)

# 
# pl.add_mesh(pvmesh, "Black", "wireframe",  opacity=0.1)

# pl.add_points(point_cloud, color="White", point_size=3.0, opacity=0.25)

# pl.add_arrows(velocity_points.points, velocity_points.point_data["V"], mag=0.0001, show_scalar_bar=True)
pl.add_mesh(pvstream, opacity=0.4, show_scalar_bar=False)
    
# pl.remove_scalar_bar("V")

# imagefile = os.path.join(image_dir, checkpoint_base) + f"_dTdr_{step}.png"

# pl.screenshot(filename=imagefile, window_size=(750, 750), return_img=False)
# OR

pl.show()
